In [1]:
import os
import numpy as np
import torch
from common.model import TemporalModel
from common.generators import UnchunkedGenerator
from common.camera import normalize_screen_coordinates
from common.utils import deterministic_random

In [2]:
keypoints_path = 'data/custom/Clip21Hit.npz'
checkpoint_path = 'checkpoint/pretrained_h36m_detectron_coco.bin'
output_keypoints_3d_path = 'outputdir/Clip21Hit3D.npz'

In [3]:
W, H = 1280, 720  # Change this if your video resolution is different
checkpoint_path = 'checkpoint/pretrained_h36m_detectron_coco.bin'
model = TemporalModel(
    num_joints_in=17, in_features=2, num_joints_out=17,
    filter_widths=[3, 3, 3, 3, 3],
    causal=False, dropout=0.25, channels=1024
)

In [4]:
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_pos'])
model.eval()
pad = (model.receptive_field() - 1) // 2

In [9]:
def keypoints3d(videoname):
    inputpath=f'data/custom/{videoname}.npz'
    outputpath=f'outputdir/{videoname}3D.npz'

    keypoints2d=np.load(inputpath)['keypoints']
    keypoints2d=normalize_screen_coordinates(keypoints2d,w=W,h=H)
    keypoints2d=keypoints2d[np.newaxis, ...]
    keypoints2d = np.pad(keypoints2d, ((0, 0), (pad, pad), (0, 0), (0, 0)), mode='edge')
    gen = UnchunkedGenerator(None, None, keypoints2d, pad=pad)

    with torch.no_grad():
        for _, _, batch_2d in gen.next_epoch():
            batch_2d = torch.from_numpy(batch_2d).float()
            predicted_3d = model(batch_2d).cpu().numpy()  # shape: (1, T, 17, 3)

    predicted_3d = predicted_3d[0]
    predicted_3d = predicted_3d[pad:-pad]
    os.makedirs(os.path.dirname(output_keypoints_3d_path), exist_ok=True)
    
    np.savez_compressed(outputpath, keypoints_3d=predicted_3d)
    print(f"[✅] Saved 3D keypoints to {outputpath} with shape {predicted_3d.shape}")

In [10]:
directory = "data/custom/"

for filename in os.listdir(directory):
    if os.path.isfile(os.path.join(directory, filename)):  # skip folders
        videoname = os.path.splitext(filename)[0]
        keypoints3d(videoname)

[✅] Saved 3D keypoints to outputdir/Clip101Miss3D.npz with shape (31, 17, 3)
[✅] Saved 3D keypoints to outputdir/Clip102Miss3D.npz with shape (35, 17, 3)
[✅] Saved 3D keypoints to outputdir/Clip103Miss3D.npz with shape (33, 17, 3)
[✅] Saved 3D keypoints to outputdir/Clip107Miss3D.npz with shape (30, 17, 3)
[✅] Saved 3D keypoints to outputdir/Clip108Miss3D.npz with shape (27, 17, 3)
[✅] Saved 3D keypoints to outputdir/Clip109Miss3D.npz with shape (27, 17, 3)
[✅] Saved 3D keypoints to outputdir/Clip110Miss3D.npz with shape (40, 17, 3)
[✅] Saved 3D keypoints to outputdir/Clip11Hit3D.npz with shape (73, 17, 3)
[✅] Saved 3D keypoints to outputdir/Clip12Hit3D.npz with shape (68, 17, 3)
[✅] Saved 3D keypoints to outputdir/Clip14Miss3D.npz with shape (68, 17, 3)
[✅] Saved 3D keypoints to outputdir/Clip16Hit3D.npz with shape (70, 17, 3)
[✅] Saved 3D keypoints to outputdir/Clip17Hit3D.npz with shape (46, 17, 3)
[✅] Saved 3D keypoints to outputdir/Clip20Miss3D.npz with shape (78, 17, 3)
[✅] Saved